# Data Science 346 Project Stellenbosch University
### Team:
- David Nicolay
- Kellen Mossner
- Matthew Holm

In [124]:
# Libraries
import pandas as pd
from transformers import pipeline, AutoTokenizer

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers

from langdetect import detect, DetectorFactory
impozrt re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Set Random seed for reproducible results
random_seed = 100 

SyntaxError: invalid syntax (2841632809.py, line 13)

In [126]:
# Additional Libraries


# Data Pre-processing

Load data:

In [130]:
# Import Data
reviews = pd.read_csv("../WebScrapingExplore/data/goodreads_reviews_all.csv")
reviews.head()

,Book Title,Link,Review Text,Review Date,Review Stars,Review Likes
0,Ways of Seeing,https://www.goodreads.com/book/show/2784.Ways_...,This book is based on a television series whic...,"September 29, 2014",5,513
1,Ways of Seeing,https://www.goodreads.com/book/show/2784.Ways_...,"I am not the audience for this book, mainly be...","June 3, 2014",3,216
2,Ways of Seeing,https://www.goodreads.com/book/show/2784.Ways_...,"Way of Seeing, John Berger Ways of Seeing is a...","October 21, 2021",4,0
3,Ways of Seeing,https://www.goodreads.com/book/show/2784.Ways_...,"First of all, this entire book is set in bold....","May 25, 2008",4,106
4,Ways of Seeing,https://www.goodreads.com/book/show/2784.Ways_...,This was a great introduction to the work of J...,"March 12, 2020",4,80


In [9]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29519 entries, 0 to 29518
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Book Title    29519 non-null  object
 1   Link          29519 non-null  object
 2   Review Text   29247 non-null  object
 3   Review Date   29519 non-null  object
 4   Review Stars  29519 non-null  int64 
 5   Review Likes  29519 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


Through investiagting our database further

In [15]:
# Function to detect language
def is_english(review):
    try:
        return detect(review) == 'en'
    except:
        return False

In [17]:
# Filter only English reviews- This may take a while
reviews = reviews[reviews['Review Text'].apply(is_english)]

In [18]:
# Preprocess the review text
def preprocess_text(text):
    if not isinstance(text, str):  # Check if the input is not a string
        text = str(text)  # Convert to string if it's not already
    
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.lower()  # Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.strip()  # Remove leading/trailing spaces
    return text


In [19]:
processed_reviews = reviews.copy()
processed_reviews['Review Text'] = reviews['Review Text'].apply(preprocess_text)

0        this book is based on a television series whic...
1        i am not the audience for this book mainly bec...
2        way of seeing john berger ways of seeing is a ...
3        first of all this entire book is set in bold i...
4        this was a great introduction to the work of j...
                               ...                        
29514    i first discovered this book when i was  and i...
29515    as per usual i loved it though i feel like the...
29516    this was actually the book that made a reader ...
29517    charlotte really put her whole brontussy into ...
29518    jane eyre was orphaned and left in the care of...
Name: Review Text, Length: 29519, dtype: object

# Part 1: Summarization

## Transformers

Initializing the pipeline will take a while to run at first, since this function downloads the model weights (about 1.6gb).

In [13]:
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
summarizer = pipeline("summarization", model=model_name)

/home/davidnicolay/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Due to restricted input length of the summarizer the reviews text needs to be divided into chunks.

In [16]:
def chunk_text(text, max_chunk_size=500):
    words = text.split()
    chunks = []
    current_chunk = []
    current_size = 0
    for word in words:
        if current_size + len(word) > max_chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_size = len(word)
        else:
            current_chunk.append(word)
            current_size += len(word) + 1  # +1 for space
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

def summarize_text(text, max_summary_length=150):
    chunks = chunk_text(text)
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=max_summary_length, min_length=10)[0]['summary_text']
        summaries.append(summary)
    
    final_summary = ' '.join(summaries)
    if len(final_summary) > max_summary_length:
        final_summary = summarizer(final_summary, max_length=max_summary_length, min_length=30)[0]['summary_text']
    return final_summary

Begin by summarizing 1 book's reviews - "Ways of Seeing"

In [35]:
# TODO change model to be able to handle longer length summarize inputs 

#book_title = "Ways of Seeing"
# book_df = reviews[reviews['Book Title'] == book_title]

# # Concatenate all reviews for the book
# all_reviews = ' '.join(book_df['Review Text'].dropna())

# # Generate a summary of the concatenated reviews
# try:
#     summary = summarize_text(all_reviews)
# except Exception as e:
#     print(f"An error occurred: {e}")
#     summary = "Error generating summary"

# # Calculate average rating
# avg_rating = book_df['Review Stars'].mean()

# # Create a new dataframe with the results
# result_df = pd.DataFrame({
#     'Book Title': [book_title],
#     'Review Summary': [summary],
#     'Average Rating': [avg_rating],
#     'Number of Reviews': [len(book_df)]
# })

# # Display the results
# print(result_df)

# # Print some statistics
# print(f"\nTotal length of all reviews: {len(all_reviews)} characters")
# print(f"Length of summary: {len(summary)} characters")

Here we can have a look at how the model does a good job of summarizing (but it essentially picks important sentences), however we still need to present it in a format that explains the overall sentiment from the readers.

In [36]:
summary_test = summarizer(book_df.loc[3]['Review Text'], max_length=50, min_length=10)

In [37]:
summary_test

[{'summary_text': '4 essays and 3 pictorial essays. It seems like museums are doing a lot of things wrong as well as right. Chapter on oil-painting was particularly interesting but it was the last one about advertising (or "publicity"'}]

In [38]:
book_df.loc[3]['Review Text']

'First of all, this entire book is set in bold. I don\'t know what crazy crazyman let that through the gate at Penguin but I just felt I had to point it out right away. It\'s still worth reading. 4 essays and 3 pictorial essays. Really interesting stuff cutting away some of the bullshit associated with our appreciation of art. It seems like museums are doing a lot of things wrong as well as right. Chapter on oil-painting was particularly interesting but it was the last one about advertising (or "publicity" as it\'s exclusively referred to in this book) that has me thinking. Advertising not only needs you to want this shirt, this car, the entire industry must endeavor to narrow the scope of your desires to make you amenable to the culture. The mindset must always be a future, better you achieved through important purchases. The essay is horrifying enough until you realise that it\'s thirty years old, and this is now only one facet of a business that\'s grown much more insidious. The ads

## Encoder-Decoder Approach

In [23]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

In [31]:

# ***************************************************************************


def preprocess(reviews):
    """
    Performs preprocessing operations such as removing new line characters.
    """
    n_reviews = len(reviews)
    print(f"Number of reviews: {n_reviews}")
    for i in range(n_reviews):
        review = reviews[i]
        # Replace newlines with spaces
        reviews[i] = review.replace('\n', ' ').strip()
        
        
def split_sentences(reviews):
    """
    Splits the reviews into individual sentences.
    """
    n_reviews = len(reviews)
    for i in range(n_reviews):
        review = reviews[i]
        import nltk
        nltk.download('punkt')
        sentences = sent_tokenize(review)  # Tokenize into sentences
        # Remove empty sentences and strip spaces
        sentences = [sent.strip() for sent in sentences if sent.strip()]
        reviews[i] = sentences
        
        
def encode_sentences(reviews):
    """
    Obtains sentence embeddings for each sentence in the reviews
    using Sentence-BERT from the sentence-transformers library.
    """
    enc_reviews = [None] * len(reviews)
    cum_sum_sentences = [0]
    sent_count = 0
    
    # Flatten reviews into a list of sentences
    for review in reviews:
        sent_count += len(review)
        cum_sum_sentences.append(sent_count)

    all_sentences = [sent for review in reviews for sent in review]
    print('Loading pre-trained Sentence-BERT model...')
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print('Encoding sentences...')
    enc_sentences = model.encode(all_sentences, show_progress_bar=True)

    # Group back encoded sentences by reviews
    for i in range(len(reviews)):
        begin = cum_sum_sentences[i]
        end = cum_sum_sentences[i+1]
        enc_reviews[i] = enc_sentences[begin:end]
        
    return enc_reviews
        
    
def summarize(reviews):
    """
    Performs summarization of book reviews.
    """
    n_reviews = len(reviews)
    summary = [None] * n_reviews
    print('Preprocessing...')
    preprocess(reviews)
    
    print('Splitting into sentences...')
    split_sentences(reviews)
    
    print('Starting to encode...')
    enc_reviews = encode_sentences(reviews)
    print('Encoding Finished')
    
    for i in range(n_reviews):
        enc_review = enc_reviews[i]
        n_clusters = int(np.ceil(len(enc_review) ** 0.5))  # Number of clusters
        
        # Perform KMeans clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans = kmeans.fit(enc_review)
        
        avg = []
        closest = []
        for j in range(n_clusters):
            idx = np.where(kmeans.labels_ == j)[0]
            avg.append(np.mean(idx))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, enc_review)
        
        # Ordering sentences by clusters
        ordering = sorted(range(n_clusters), key=lambda k: avg[k])
        summary[i] = ' '.join([reviews[i][closest[idx]] for idx in ordering])
    
    print('Clustering Finished')
    return summary


#### What is happening here?
Encoding:
- Sentence-BERT: The function uses a pre-trained Sentence-BERT model ('all-MiniLM-L6-v2') to convert each sentence into a vector embedding. - This embedding is a numerical representation of the sentence that captures its semantic meaning.
- It first flattens all the sentences from the reviews into a single list and then encodes them.
- After encoding, it restructures the embeddings back into their original review groups.

Clustering:
- For each review, the number of clusters is determined using the square root of the number of sentences (rounded up).
- KMeans clustering is performed on the sentence embeddings to group similar sentences.
- For each cluster, **the sentences closest to the cluster center (based on the distance between sentence embeddings) are selected.**

### Applying to a single book's review

In [132]:
reviews_subset = reviews[reviews['Book Title'] == 'Ways of Seeing']

In [134]:
way_of_seeing_reviews = reviews_subset['Review Text'].tolist()
summaries = summarize(way_of_seeing_reviews)

Preprocessing...
Number of reviews: 120
Splitting into sentences...
Starting to encode...
Loading pre-trained Sentence-BERT model...
Encoding sentences...


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Encoding Finished
Clustering Finished


In [35]:
summaries[1]

"In general, the lesson of this book is that all art is bad for you, except the pieces that the authors of this book like. I am not the audience for this book, mainly because I've already read and more or less digested the handful of essays and ideas on which it is based. This is true. They don't discuss the 20th century at all (I know they know that twentieth century art exists; perhaps, as good Benjaminian Marxists, they don't like abstraction or difficulty). Holbein's 'Ambassadors' is read as an example of this; the incredible distorted skull in the painting is the exception which proves the rule of oil paintings rather than, you know, showing that oil paintings can be self-critical, as are most good artworks of any kind. Harmful because those who accept it will say silly things, and because those who read it and reject it out of hand (due to the rhetoric, bad arguments, or conceptual confusion) won't be challenged to, you know, care about other people."

In [37]:
reviews['Review Text'][1]

'I am not the audience for this book, mainly because I\'ve already read and more or less digested the handful of essays and ideas on which it is based. The seven chapters break down fairly simply. 1: Benjamin\'s \'Work of Art\'--the ability to reproduce images alters the way we encounter works of art. This seems reasonable. Nobody gets to see a Giotto without having seen a reproduction first, except someone who has no interest in the Giotto in the first place. But Berger et al* go a step further: we need to use the fact that we encounter works of art differently to undermine the ruling class\'s privilege and the "specialized experts who are the clerks of the nostalgia of a ruling class in decline." That\'s on page 32. Part of me, a large part, laments the fact that you\'d never get that published today, not even on a website. Another part of me laments the stupidity of intellectuals who put their faith in the inherent goodness of The People. The People does not have a good track record

## Same Approach as in class

## TODO

#### First Apply to 1 book for example sake

In [63]:
import warnings
warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows with MKL")

In [65]:
born_a_crime_subset = reviews[reviews['Book Title'] == 'Born a Crime: Stories From a South African Childhood']
born_a_crime_reviews = born_a_crime_subset['Review Text'].tolist()
born_a_crime_summaries = summarize(born_a_crime_reviews)

Preprocessing...
Number of reviews: 120
Splitting into sentences...
Starting to encode...
Loading pre-trained Sentence-BERT model...
Encoding sentences...


Batches:   0%|          | 0/67 [00:00<?, ?it/s]

Encoding Finished
Clustering Finished


In [89]:
born_a_crime_summaries[45]


"Noah sums it up very well: in America we had 1) forced removal of the native population to reservations, 2) slavery, followed by 3) segregation. Like in the best books, you learn some important history in the telling of his young life's story. One of the many things I didn't know about South Africa is that as most countries were trying to fix racist policies after the World War 2 holocaust had shown us where discrimination could lead, the South Africans or Afrikaners (as the Dutch colonists called themselves) were running towards institutionalized racism. This man's talent knows no ceiling."

In [91]:
born_a_crime_reviews[45]

["One of the best memoirs I've ever read, Born a Crime is so funny and wise and sad and reveals much about the writer.",
 "Trevor Noah is an exceptional young man raised by a strong, formidable mother who never gave up on him and loved him with a fierce pride.He never shrinks from complete honesty in the telling, even in those areas that don't reflect well on his teenaged and early adult self.",
 'One very amusing incident tells of how as a kid his rejection of going to the outside toilet in the rain led to a most unfortunate incident with his blind grandma.',
 "Like in the best books, you learn some important history in the telling of his young life's story.",
 'Apartheid was made officially part of South African government in 1948; whereas, before you had casual, implied racism, now it was a system of specific laws that rated you as a person.',
 'There were different rules for whites, colored (people descended from the first white settlers and the natives), blacks, Indians and Asians

So far so good - the summaries skip over some information here and there, but at a glance, they look quite accurate. Now let's take it a step further and generate a final summary that should encapsulate the core themes and ideas mentioned in the reviews.

In [93]:
def summarize_final(all_summaries):
    """
    Performs summarization of the combined summaries.
    """
    all_summaries = [summary.replace('\n', ' ').strip() for summary in all_summaries]
    
    from nltk.tokenize import sent_tokenize
    all_sentences = []
    for summary in all_summaries:
        sentences = sent_tokenize(summary)
        sentences = [sent.strip() for sent in sentences if sent.strip()]
        all_sentences.extend(sentences)
    
    print('Loading pre-trained Sentence-BERT model...')
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print('Encoding sentences...')
    enc_sentences = model.encode(all_sentences, show_progress_bar=True)

    n_clusters = int(np.ceil(len(enc_sentences) ** 0.5))
    print('Clustering sentences...')
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans = kmeans.fit(enc_sentences)
    
    avg = []
    closest = []
    for j in range(n_clusters):
        idx = np.where(kmeans.labels_ == j)[0]
        avg.append(np.mean(idx))
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, enc_sentences)
    
    ordering = sorted(range(n_clusters), key=lambda k: avg[k])
    final_summary = ' '.join([all_sentences[closest[idx]] for idx in ordering])
    
    print('Final summary generated')
    return final_summary

In [118]:
def generate_book_description(summary):
    """
    Takes the final summary of reviews and generates a book description.
    """
    description_pipeline = pipeline("text2text-generation", model="facebook/bart-large-cnn")

    prompt = (
        "Convert the following summary into a structured book description that describes the book's content, "
        "themes, and significance: " + summary
    )

    result = description_pipeline(prompt, max_length=250, min_length=100, do_sample=False)

    book_description = result[0]['generated_text']
    #print(book_description)
    return book_description

In [99]:
final_born_a_crime_summary = summarize_final(born_a_crime_summaries)

Loading pre-trained Sentence-BERT model...
Encoding sentences...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Clustering sentences...
Final summary generated


In [101]:
final_born_a_crime_summary

"He was just Robert to Trevor, someone who was nice to him on Sundays and Christmas holidays, and on all those days when he was not giving in to hungry black women his Swiss, precious, and fair sperm. If you have a chance to listen to the book, instead of reading it, I recommend this experience highly. Noah's love and respect for his mother & the way she raised him shines through on nearly every page. Eye-opening and perspective changing in a way that's funny and deeply vulnerable, you'll feel educated and entertained at the same time. Trevor Noah is a remarkable person. I LOVED this book! I liked reading about his childhood. But Trevor Noah expertly maneuvers through different scenes--much of it due to his mixed background--and explains so much history with apartheid and the different tribes and the colored thing and life in the townships and life in the homeland and so on... South Africa does of course have it's problems, yet after this book one has to be at least more empathetic. . 

Ignoring the **extremely** odd first sentence, this final generated summary mentions essentially every aspect that you can find in most reviews (audiobook being better, Noah's relationship with his mother, the premise, etc). For the sake of coherency, we can now parse this final summary to `facebook/bart-large-cnn` with a prompt to reframe the summary as a book description.

`facebook/bart-large-cnn` is a fine-tuned version of BART (Bidirectional and Auto-Regressive Transformer), a transformer-based sequence-to-sequence model introduced by Facebook AI, combining the advantages of bidirectional and auto-regressive models. The model uses an encoder-decoder architecture: the encoder processes the entire input (like BERT), and the decoder generates output token-by-token (like GPT).

Pretrained on various denoising tasks to learn language patterns, BART was fine-tuned specifically on the CNN/DailyMail dataset to improve its summarization capabilities. During fine-tuning, the decoder attends to the encoder's output to generate contextually appropriate summaries (Attention Mechanisms). Beam search and length penalty were also used to avoid overly short summaries.

The weights were initialized from the original BART model that was pretrained on a large corpus of text using the denoising autoencoding tasks mentioned earlier.



In [120]:
book_description = generate_book_description(final_born_a_crime_summary)

In [109]:
book_description

"Trevor Noah's love and respect for his mother & the way she raised him shines through on nearly every page. Eye-opening and perspective changing in a way that's funny and deeply vulnerable, you'll feel educated and entertained at the same time. For an enhanced experience, I highly recommend the audiobook version. Moved out of the house at the age of 17 because of his step-dad and was even jailed for using a fake license plate. Imagine being born from a black mother and a white father in a country where interracial relationships were against the law."

We can now apply this to any book we like to obtain an overview/description of the contents and themes soley based off of the reviews.

# Part 2: Manual Deep Learning

## Star Prediction

In [27]:
# Setup target and predictor datasets
X = reviews['Review Text'].values
y = reviews['Review Stars'].values

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_tfidf = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=random_seed)

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [40]:
# Build the model
model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=1)

Epoch 1/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.4730 - loss: 1.2667 - val_accuracy: 0.5401 - val_loss: 1.0601
Epoch 2/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6598 - loss: 0.8296 - val_accuracy: 0.5418 - val_loss: 1.1059
Epoch 3/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8133 - loss: 0.5163 - val_accuracy: 0.5057 - val_loss: 1.3474
Epoch 4/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9334 - loss: 0.2233 - val_accuracy: 0.5001 - val_loss: 1.9749
Epoch 5/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9863 - loss: 0.0553 - val_accuracy: 0.5105 - val_loss: 2.6528
Epoch 6/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9957 - loss: 0.0185 - val_accuracy: 0.4947 - val_loss: 3.1868
Epoch 7/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9985 - loss: 0.0062 - val_accuracy: 0.4968 - val_loss: 3.4601
Epoch 8/10
505/505 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9983 - loss: 0.0061 - val_accuracy: 0.

In [41]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test.toarray(), y_test, verbose=0)
print(f'Test accuracy: {test_accuracy:.3f}')

Test accuracy: 0.481


A test accuracy below 0.5 means we could have predicted 2.5 stars for every review and probably gotten beter results. The training accuracy seems to increase drastically however validation accuracy remains steady at around 0.51. This indicates overfitting.

TODO
- metric graphs
- improve model
- look online for differnet approachs and things to do to improve
- try gradient descent etc

### Example Predictions

Consider an example of a bad review. "This book was absolutely terrible! How could you think this was a good idea."

In [31]:
# Make predictions
sample_review = ["This book was absolutely terrible! How could you think this was a good idea."]
sample_review_tfidf = vectorizer.transform(sample_review)
prediction = model.predict(sample_review_tfidf.toarray())
predicted_rating = np.argmax(prediction) + 1
print(f'Predicted rating: {predicted_rating}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predicted rating: 1


Consider an example of a long mixed review

In [32]:
sample_review = ["I had high hopes for The Infinite Horizon after hearing so much about it. From the beginning, the premise seemed promising, and for the most part, it delivers on its intriguing concept. The plot revolves around a futuristic world where society grapples with the boundaries of artificial intelligence, humanity, and survival—concepts that have always fascinated me. The world-building is impressive, with detailed landscapes and a unique societal structure that keeps you hooked initially. The author has clearly put a lot of thought into constructing the futuristic world, and it shows in the vivid descriptions and creative technologies. However, while the world-building is rich, the characters left much to be desired. The protagonist, Lila, felt underdeveloped. I found myself frustrated at several points because her motivations were either unclear or inconsistent. In the beginning, she starts off as a strong, determined character, but midway through, her actions seem erratic and her growth stagnates. The dialogue, too, felt stilted at times, making it hard to connect with the characters emotionally. There were a few moments where I felt the conversations between key characters were forced, almost like they were inserted to explain plot points rather than feeling organic.On the flip side, I have to give credit where it’s due—the pacing of the story is solid for the most part. There are intense moments where you’re on the edge of your seat, particularly during the battle scenes. These scenes were written with such vivid detail that I could easily imagine them playing out in a movie. The action sequences are well thought out, and they definitely add excitement to the narrative. That being said, there were also moments where the pacing lagged, especially in the middle sections. Some chapters felt like filler, dragging on with unnecessary exposition and side plots that didn’t add much to the overarching story."]
sample_review_tfidf = vectorizer.transform(sample_review)
prediction = model.predict(sample_review_tfidf.toarray())
predicted_rating = np.argmax(prediction) + 1
print(f'Predicted rating: {predicted_rating}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted rating: 3


# Part 3: Category Generation